In [1]:
import torch
print(torch.__version__)

1.8.1+cu102


In [2]:
# Run this cell to see if things work
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [3]:
# Basic tools of private
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


#  ```x_ptr = tensor.send(bob)```to send
#  ```x_ptr.get()``` to get back from the virtual worker

In [5]:
#Sending Tensors to Bob's Machine
bob = sy.VirtualWorker(hook, id="bob")

AttributeError: module 'syft' has no attribute 'VirtualWorker'

In [ ]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

In [ ]:
me = sy.local_worker
me

In [ ]:
me._objects

In [ ]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [ ]:
me._objects

In [ ]:
x_ptr

In [ ]:
bob._objects

In [ ]:
z = x_ptr + x_ptr

In [ ]:
z

In [ ]:
bob._objects

In [ ]:
###############
x_ptr

In [ ]:
x_ptr.location

In [ ]:
#
x_ptr.id_at_location

In [ ]:
x_ptr.id

In [ ]:
x_ptr.location
#############

In [ ]:
me == x_ptr.owner

# ``` get ```

In [ ]:
x_ptr.get()
y_ptr.get()
z.get()

In [ ]:
me._objects

In [ ]:
bob._objects# empty

# Section1.2 Using Tensor Pointers

In [ ]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [ ]:
z = x + y

In [ ]:
z

在幕后，发生了一件非常强大的事情。不是 x 和 y 在本地计算加法，而是将命令序列化并发送给 Bob，Bob 执行计算，创建张量 z

In [ ]:
bob._objects

# Torch Functions

In [ ]:
x

In [ ]:
y

In [ ]:
z = torch.add(x,y)
z

In [ ]:
bob._objects

In [ ]:
z.get()

In [ ]:
bob._objects

In [ ]:
#z.get()

In [ ]:
bob._objects

# Variables (including backpropagation!)

In [ ]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [ ]:
x.grad

In [ ]:
(x + y).sum()

In [ ]:
z = (x + y).sum()
z

In [ ]:
bob._objects

In [ ]:
z.backward()

In [ ]:
x = x.get()
x

In [ ]:
x.grad

#  Part 2: Intro to Federated Learning


Optimizer and training(Only pytorch)

In [ ]:
import torch
from torch import nn
from torch import optim
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
train()

Optimizer and training(federated learning optimizer) 
  
    *create a couple workers  
    *get pointers to training data on each worker  
    *updated training logic to do federated learning  
New Training Steps:  
  
    *send model to correct worker  
    *train on the data located there  
    *get the model back and repeat with next  

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)

In [ ]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")### Optimizer

### Data set

In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
#torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)

# in_features – size of each input sample
# out_features – size of each output sample
model = nn.Linear(2,1)

#all pointers
data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

### Informations

In [ ]:
data, target = datasets[0]
data

In [ ]:
data.owner

In [ ]:
data.id

In [ ]:
data.location

In [ ]:
data.id_at_location

### Optimizer

In [ ]:
from syft.federated.floptimizer import Optims #跟pytorch optimis 类似
workers = ['bob', 'alice']
optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.1))#参考pytorch### Optimizer

### Train
send model to the VirtualWorker
train the model
get model from the VirtualWorker

In [ ]:
#only model trainings at Alice and Bob  privacy not protected 
def train():
    # Training Logic
    for iter in range(1):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            # 'tensor' to be sent to the VirtualWorker. model = tensor.nn.Linear
            model.send(data.location)
            
            #Call the optimizer for the worker using get_optim
            opt = optims.get_optim(data.location.id)
            #print("data.location.id", data.location.id)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            # return `tensor` from VirtualWorker and delete it frome the VirtualWorker
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [ ]:
train()

# Part 3: Advanced Remote Execution Tools
we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

In [ ]:
# import torch
# import syft as sy
# hook = sy.TorchHook(torch)

pointers to pointers

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [ ]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

In [ ]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

In [ ]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

In [ ]:
bob._objects

In [ ]:
alice._objects